In [1]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
import catboost as cb
from catboost import CatBoostClassifier
import time

from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, f1_score, roc_auc_score, average_precision_score, precision_score, recall_score, classification_report
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split, StratifiedKFold, cross_val_score

from cc5_preprocessor import Preprocessor

import joblib

np.random.seed(42)

In [2]:
df = pd.read_csv('data/Training_TriGuard.csv')
df = df.dropna(subset=['subrogation'])

In [3]:
pre = Preprocessor(smoothing_factor=5, mode = 'catboost')

In [4]:
X = df.drop(columns=["subrogation"]).copy()
y = df["subrogation"].copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state=42)

In [5]:
pre.fit(X_train, y_train)

X_train_proc = pre.transform(X_train)
X_test_proc = pre.transform(X_test)

X_test_proc = X_test_proc.reindex(columns=X_train_proc.columns, fill_value=0)

Fitting Preprocessor in 'catboost' mode...
CatBoost mode: Skipping target encoding learning.
Learning statistical parameters for Z-scoring...
Fit complete.
Transforming data in 'catboost' mode...
Applying learned statistical transforms (Z-scores)...
CatBoost mode: Skipping target encoding application.
CatBoost mode: Dropping unused object/datetime columns...
Dropping: ['witness_present_ind', 'claim_date']
Transform complete.
Transforming data in 'catboost' mode...
Applying learned statistical transforms (Z-scores)...
CatBoost mode: Skipping target encoding application.
CatBoost mode: Dropping unused object/datetime columns...
Dropping: ['witness_present_ind', 'claim_date']
Transform complete.


In [6]:
pre.fit(X, y)
X_proc = pre.transform(X)

Fitting Preprocessor in 'catboost' mode...
CatBoost mode: Skipping target encoding learning.
Learning statistical parameters for Z-scoring...
Fit complete.
Transforming data in 'catboost' mode...
Applying learned statistical transforms (Z-scores)...
CatBoost mode: Skipping target encoding application.
CatBoost mode: Dropping unused object/datetime columns...
Dropping: ['witness_present_ind', 'claim_date']
Transform complete.


## CatBoost with Optuna Tuning

In [7]:
import optuna
from optuna.integration import CatBoostPruningCallback

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
CAT_FEATURES = pre.cat_for_encoding_
print(CAT_FEATURES)

['accident_site', 'accident_type', 'channel', 'vehicle_category', 'vehicle_color', 'living_status', 'claim_day_of_week', 'gender', 'in_network_bodyshop', 'season']


In [9]:


def objective(trial: optuna.trial.Trial) -> float:
    
    # 1. Define hyperparameters from Optuna
    params = {
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'depth': trial.suggest_int('depth', 3, 10),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-3, 10.0, log=True),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0, step=0.1),
        'random_strength': trial.suggest_float('random_strength', 1e-8, 1.0, log=True),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 1.0),
        'border_count': trial.suggest_int('border_count', 32, 255),
        'scale_pos_weight': trial.suggest_float('scale_pos_weight', 1.0, 10.0),
        
        # --- THIS IS THE FIX ---
        # Add the static parameters (including cat_features) here
        'eval_metric': 'F1',
        'task_type': 'CPU',
        'verbose': False,
        'random_state': 42,
        'cat_features': CAT_FEATURES  # <-- Pass cat_features at initialization
    }

    # 2. Set up the model
    # The model now knows about cat_features from the start
    model = CatBoostClassifier(**params)
    
    # 3. Configure CV Splitter
    skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42) # 3-fold for speed

    # 4. Run CV on the TRAINING data
    # We no longer need 'fit_params' because the model already has the info
    f1_scores = cross_val_score(
        model,
        X_proc,
        y,
        cv=skf,
        scoring='f1'
    )
    
    # 5. Return the mean F1 from the folds
    return np.mean(f1_scores)

In [10]:
print("\n2. Starting Optuna study...")

study = optuna.create_study(
    direction='maximize',
    pruner=optuna.pruners.MedianPruner(n_warmup_steps=10)
)

study.optimize(
    objective, 
    n_trials=100, # Number of trials to run
    show_progress_bar=True
)

print("\n" + "="*50)
print("Optuna study finished.")
print(f"Number of finished trials: {len(study.trials)}")

print("\nBest trial:")
best_trial = study.best_trial
    
print(f"  Value (Max F1 Score): {best_trial.value:.4f}") # <-- CHANGED comment
    
print("  Best Hyperparameters:")
for key, value in best_trial.params.items():
    print(f"    {key}: {value}")

[I 2025-11-12 10:08:43,963] A new study created in memory with name: no-name-58900435-52e2-4174-be36-6c897c75d257



2. Starting Optuna study...


Best trial: 0. Best value: 0.478358:   1%|          | 1/100 [01:09<1:54:46, 69.56s/it]

[I 2025-11-12 10:09:53,527] Trial 0 finished with value: 0.4783576975191634 and parameters: {'learning_rate': 0.1979972206019883, 'depth': 9, 'l2_leaf_reg': 0.18697203111532762, 'subsample': 0.6, 'random_strength': 6.678003128126955e-06, 'bagging_temperature': 0.6468636251717107, 'border_count': 144, 'scale_pos_weight': 1.2195794316456112}. Best is trial 0 with value: 0.4783576975191634.


Best trial: 1. Best value: 0.573883:   2%|▏         | 2/100 [01:26<1:02:58, 38.56s/it]

...

[I 2025-11-12 10:44:46,750] Trial 97 finished with value: 0.587752278481268 and parameters: {'learning_rate': 0.011720156965360239, 'depth': 3, 'l2_leaf_reg': 6.836489660363154, 'subsample': 0.8, 'random_strength': 5.225542608283866e-08, 'bagging_temperature': 0.6565914774868797, 'border_count': 32, 'scale_pos_weight': 3.010175180611438}. Best is trial 77 with value: 0.5921506942035311.


Best trial: 77. Best value: 0.592151:  99%|█████████▉| 99/100 [36:16<00:13, 13.76s/it]

[I 2025-11-12 10:45:00,243] Trial 98 finished with value: 0.5502452429842166 and parameters: {'learning_rate': 0.01432146887622977, 'depth': 3, 'l2_leaf_reg': 4.901361719929764, 'subsample': 0.9, 'random_strength': 7.756657129215044e-08, 'bagging_temperature': 0.6054161131109497, 'border_count': 45, 'scale_pos_weight': 1.269444052136957}. Best is trial 77 with value: 0.5921506942035311.


Best trial: 77. Best value: 0.592151: 100%|██████████| 100/100 [36:29<00:00, 21.90s/it]

[I 2025-11-12 10:45:13,581] Trial 99 finished with value: 0.5917327188203472 and parameters: {'learning_rate': 0.012951213885885838, 'depth': 3, 'l2_leaf_reg': 3.742858121642672, 'subsample': 0.8, 'random_strength': 1.7260125946577633e-08, 'bagging_temperature': 0.5428827851299092, 'border_count': 67, 'scale_pos_weight': 2.135399749502426}. Best is trial 77 with value: 0.5921506942035311.

Optuna study finished.
Number of finished trials: 100

Best trial:
  Value (Max F1 Score): 0.5922
  Best Hyperparameters:
    learning_rate: 0.010187778992423046
    depth: 4
    l2_leaf_reg: 4.7768640731726455
    subsample: 0.8
    random_strength: 7.470999613751303e-08
    bagging_temperature: 0.5691931351789441
    border_count: 103
    scale_pos_weight: 2.3973369442371877


In [11]:
best_params = study.best_trial.params
print(best_params)

final_params = best_params.copy()
final_params.update({
    'iterations': 2000, # Use more iterations for the final model
    'eval_metric': 'Logloss', # Use Logloss for training/stopping
    'task_type': 'CPU',
    'early_stopping_rounds': 100, # Keep early stopping
    'random_state': 123
})

best_model = CatBoostClassifier(**final_params)

best_model.fit(
    X_train_proc, y_train,
    eval_set=(X_test_proc, y_test),
    cat_features=CAT_FEATURES,
    verbose=False
)

print(f"\nFinal Model Score (from best Logloss iteration):")
y_preds_final = best_model.predict(X_test_proc)
final_f1 = f1_score(y_test, y_preds_final, pos_label=1)
print(f"  Manual F1:class=1 Score: {final_f1:.4f}")
        
print("\n  Full Classification Report:")
print(classification_report(y_test, y_preds_final, target_names=['Class 0.0', 'Class 1.0']))

{'learning_rate': 0.010187778992423046, 'depth': 4, 'l2_leaf_reg': 4.7768640731726455, 'subsample': 0.8, 'random_strength': 7.470999613751303e-08, 'bagging_temperature': 0.5691931351789441, 'border_count': 103, 'scale_pos_weight': 2.3973369442371877}

Final Model Score (from best Logloss iteration):
  Manual F1:class=1 Score: 0.5830

  Full Classification Report:
              precision    recall  f1-score   support

   Class 0.0       0.90      0.80      0.84      4165
   Class 1.0       0.50      0.70      0.58      1235

    accuracy                           0.77      5400
   macro avg       0.70      0.75      0.71      5400
weighted avg       0.81      0.77      0.78      5400



In [15]:
print("Saving best_model...")

# Use the model's F1 score in the name
best_model.save_model("catboost_mod_f1_0.6020.cbm")

print("Done.")

Saving best_model...
Done.


In [16]:
importances = best_model.get_feature_importance()
feature_names = best_model.feature_names_

feature_importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance': importances
})

feature_importance_df = feature_importance_df.sort_values(by='importance', ascending=False)

print("--- Feature Importance Analysis ---")
print(f"Total features: {len(feature_importance_df)}")

print("\nTop 10 Most Important Features:")
print(feature_importance_df.head(10))

print("\nBottom 10 Least Important Features:")
print(feature_importance_df.tail(10))

n_features_to_remove = 100
n_features_to_keep = len(feature_importance_df) - n_features_to_remove
top_features = feature_importance_df.head(n_features_to_keep)['feature'].tolist()

print(f"\nKeeping top {len(top_features)} features and removing bottom {n_features_to_remove}.")

X_train_top_features = X_train_proc[top_features]
X_test_top_features = X_test_proc.reindex(columns=top_features, fill_value=0) 

original_cat_features = set(CAT_FEATURES)
top_features_set = set(top_features)
new_cat_features = list(original_cat_features.intersection(top_features_set))

print(f"Original categorical features: {len(CAT_FEATURES)}")
print(f"Categorical features kept: {len(new_cat_features)}")

best_params_from_optuna = study.best_trial.params

final_params_new = best_params_from_optuna.copy()
final_params_new.update({
    'iterations': 2000, 
    'eval_metric': 'Logloss',
    'task_type': 'CPU',
    'early_stopping_rounds': 50
})

print("\nRetraining model with top features...")
new_model = CatBoostClassifier(**final_params_new)

new_model.fit(
    X_train_top_features, y_train,
    eval_set=(X_test_top_features, y_test),
    cat_features=new_cat_features,
    verbose=False
)

y_preds_new = new_model.predict(X_test_top_features)
new_f1 = f1_score(y_test, y_preds_new, pos_label=1)

print("\n--- Model Performance Comparison ---")
# 'final_f1' comes from cell 30 in your notebook
print(f"Original F1 score (all features): {final_f1:.4f}")
print(f"New F1 score (top {len(top_features)} features): {new_f1:.4f}")

print("\nNew Model Classification Report (Top Features):")
print(classification_report(y_test, y_preds_new, target_names=['Class 0.0', 'Class 1.0']))

--- Feature Importance Analysis ---
Total features: 161

Top 10 Most Important Features:
                        feature  importance
58            liab_prct_squared   14.854113
152  recovery_feasibility_score   10.996216
62                 liab_inverse    8.988131
61                liab_prct_log    6.934850
60               liab_prct_sqrt    5.346997
63         liab_inverse_squared    5.229173
51           witness_x_multicar    4.556612
46                liab_x_police    3.137194
53          multicar_x_highrisk    2.836996
59              liab_prct_cubed    2.705797

Bottom 10 Least Important Features:
                feature  importance
107   very_high_mileage         0.0
97    mid_price_vehicle         0.0
105        high_mileage         0.0
104  is_compact_vehicle         0.0
42        police_binary         0.0
102   is_medium_vehicle         0.0
101       medium_weight         0.0
100       light_vehicle         0.0
98      economy_vehicle         0.0
160     annual_income_z       

In [17]:
# Output module, from model_citizens.ipynb
real_test = pd.read_csv("data/Testing_TriGuard.csv")

X_real_test_proc = pre.transform(real_test)
X_real_test_proc = X_real_test_proc.reindex(columns=X_train_proc.columns, fill_value=0)
real_pred_proba = best_model.predict_proba(X_real_test_proc)[:, 1]
real_pred_label = (real_pred_proba >= 0.5).astype(int)

prediction = pd.DataFrame({
    "claim_number": real_test["claim_number"],
    "subrogation": real_pred_label
})

print(prediction.head())

Transforming data in 'catboost' mode...
Applying learned statistical transforms (Z-scores)...
CatBoost mode: Skipping target encoding application.
CatBoost mode: Dropping unused object/datetime columns...
Dropping: ['witness_present_ind', 'claim_date']
Transform complete.
   claim_number  subrogation
0       3126034            0
1       7380142            0
2       4655051            0
3       6728725            1
4       9848460            1


In [18]:
prediction.to_csv("results/catboost_6020_prediction.csv", index=False)